# DALLE-3

In [1]:
import os

os.chdir("../../")

In [2]:
from langchain.chat_models import ChatOpenAI

from src.initialization import credential_init


credential_init()

model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                   model_name="gpt-4o-2024-05-13", temperature=0)

C:\Users\MengChieh\Miniconda3\envs\llm_examples\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


### OpenAI Image API Parameters:

- model: dall-e-3
- size (str): 1024x1024, 1024x1792, 1792x1024
- quality: hd, standard
- style: vivid, natural. Default vivid

In [3]:
from openai import OpenAI

prompt = """
A Sumi-e style watercolor painting of mountains during sunset. The sky is depicted with bold splashes of orange, pink, and purple hues, 
blending and overlapping in a dynamic composition. The mountains are represented with expressive brushstrokes, emphasizing their majestic and serene presence. 
The focus is on capturing the essence and mood of the scene rather than detailed realism. The overall effect is serene and contemplative, with a harmonious 
balance of color and form.
"""

client = OpenAI()

response = client.images.generate(
    model="dall-e-3",
    prompt=prompt,
    size="1024x1024",
    quality="hd",
    n=1,
    response_format = 'b64_json'
)

image_base64 = response.data[0].b64_json

## Save the image in your local computer

In [4]:
import base64

with open("tutorial/Week-8/test.png", "wb") as fh:
    fh.write(base64.b64decode(image_base64))

## Two Challenges:

### 1. How to create prompt more efficiently? 

There are two types of prompt: 

1. Danbooru Tag: masterpiece, best quality, beautiful eyes, clear eyes, detailed eyes, Blue-eyes, 1girl, 20_old, full-body, break, smoking, break, high_color, blue-hair, beauty, black-boots,break, break, Flat vector art, Colorful art, white_shirt, simple_background, blue_background, Ink art, peeking out upper body, Eyes

2. Natural language: A Sumi-e style watercolor painting of mountains during sunset. The sky is depicted with bold splashes of orange, pink, and purple hues, blending and overlapping in a dynamic composition. The mountains are represented with expressive brushstrokes,emphasizing their majestic and serene presence. The focus is on capturing the essence and mood of the scene rather than detailed realism. The overall effect is serene and contemplative, with a harmonious balance of color and form.

As non-native English speakers, we find the natural language prompt challenging, even for native speakers, due to the inclusion of specialized terminologies and advanced vocabulary.

由於涉及專業術語和高級詞彙，我們作為非母語英語使用者，發現這個自然語言提示對我們來說是具有挑戰性的，即使對母語使用者來說也是如此。

### 2. How to make it an LCEL?

## Some websites for natural language prompt

- https://leonardo.ai/: An Image generation SaaS. A lot of works are created with natural language prompt. 
- https://blog.mlq.ai/dalle-prompts/: Some tutorial about how to come up with a natural language prompt.

### Natural Language Prompt Generation

In [5]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser


system_prompt = PromptTemplate.from_template('''You are a helpful AI assistant and an art expert with extensive knowledge of photography and illustration. 
You excel at creating breathtaking masterpieces with the DALLE-3 model. For this task, you will be provided with a description of an image, and you will 
generate a corresponding DALLE-3 prompt. The prompt should be detailed and descriptive, capturing the essence of the image. The length of the prompt should be 
around 100-500 tokens.''')

# System prompt
system_message = SystemMessagePromptTemplate(prompt=system_prompt)

human_prompt = PromptTemplate(template="{image_desc}",
                              input_variables=['image_desc'])

# Create a human message prompt template based on the prompt
human_message = HumanMessagePromptTemplate(prompt=human_prompt)

chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

nl_prompt_generation_chain = chat_prompt | model | StrOutputParser()

## We wrap OPENAI API call as a function for the langchain usage 

In [6]:
from typing import Dict


def dalle3_worker(kwargs: Dict):

    """
    Generates an image using OpenAI's DALL-E 3 model based on the provided prompt and optional parameters.
    
    Parameters:
    kwargs (Dict): A dictionary containing the following keys:
        - 'nl_prompt' (str): The natural language prompt describing the image to be generated.
        - 'size' (str, optional): The size of the generated image. Default is "1024x1024".
        - 'quality' (str, optional): The quality of the generated image. Default is "standard".
        - 'style' (str, optional): The style of the generated image. Default is "vivid".
    
    Returns:
    str: The URL of the generated image.
    
    Example:
    >>> kwargs = {
    ...     "nl_prompt": "A futuristic city skyline at sunset",
    ...     "size": "1024x1024",
    ...     "quality": "high",
    ...     "style": "photorealistic"
    ... }
    >>> image_base64 = dalle3_worker(kwargs)
    Start generating image...
    prompt: A futuristic city skyline at sunset
    generated_image: https://example.com/generated_image.png
    >>> print(image_base64)
    https://example.com/generated_image.png
    """
    
    print("Start generating image...")
    print(f"prompt: {kwargs['nl_prompt']}")
    client = OpenAI()

    response = client.images.generate(
        model="dall-e-3",
        prompt=kwargs['nl_prompt'],
        size=kwargs.get("size", "1024x1024"),
        quality=kwargs.get('quality', 'standard'),
        style=kwargs.get('style', 'vivid'),
        n=1,
        response_format = 'b64_json')

    image_base64 = response.data[0].b64_json

    print("Image is generated succesfully.")
    
    return image_base64


def base64_to_file(kwargs):

    """
    Save the image from a base64 string
    """
    
    image_base64 = kwargs['image_base64']
    filename = kwargs['filename']
    
    with open(f"{filename}", "wb") as fh:
        fh.write(base64.b64decode(image_base64))
    

In [7]:
from operator import itemgetter

from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough

# step 1: create the image prompt
step_1 = RunnablePassthrough.assign(nl_prompt=itemgetter('image_desc')|nl_prompt_generation_chain)

# step 2: image generation process, as a base64
step_2 = RunnablePassthrough.assign(image_base64=dalle3_worker)

# step 3: save the image
step_3 = RunnableLambda(base64_to_file)

# chain step 1, step 2, step 3 together
dalle3_chain =  step_1|step_2|step_3

In [8]:
dalle3_chain.invoke({"size": "1024x1792",
                     "quality": "hd",
                     "image_desc": """
                                     masterpiece, best quality, beautiful eyes, clear eyes, detailed eyes, Blue-eyes, 1girl, 20_old, full-body, 
                                     break, smoking, break, high_color, blue-hair, beauty, black-boots,break, break, Flat vector art, Colorful art, white_shirt, 
                                     simple_background, blue_background, Ink art,peeking out upper body,Eyes, portrait
                                     """,
                     
                     "filename": "tutorial/Week-8/test_01.png"
                    })

Start generating image...
prompt: Create a stunning flat vector art masterpiece featuring a 20-year-old woman with striking blue eyes and detailed, clear eyes. She has vibrant blue hair and is depicted in a full-body pose, taking a break and smoking. The scene is colorful and high in contrast, with the woman wearing a white shirt and black boots. The background is simple and blue, allowing the subject to stand out. The art style is a blend of flat vector and ink art, with a focus on the beauty and expressiveness of her eyes. The composition includes a portrait-like peeking out of her upper body, emphasizing her beautiful eyes and serene expression.
Image is generated succesfully.


In [9]:
dalle3_chain.invoke({"size": "1024x1792",
                     "quality": "hd",
                     "image_desc": """
                                 close-up portrait, black fox ears, animal ear fluff, black fox tail, black hair, red inner hair, hair ornament, 
                                 magatama necklace, fur trim, black short kimono, exquisite design, cat_collar, off-shoulder,wide sleeves, 
                                 long sleeves, obi, miniskirt, perfect model body, a 17-years-old ethereal and glamorously beautiful girl, from above, 
                                 eating donut, holding a donut, a large cup of coffee on table, in a coffee shop, pencil sketch, perfect detail, intricate detail, 
                                 masterpiece, best quality, beauty & aesthetic, sketch
                                 """,
                     "filename": "tutorial/Week-8/test_02.png"
                    })


Start generating image...
prompt: A pencil sketch of an ethereal and glamorously beautiful 17-year-old girl with a perfect model body, captured from above in a coffee shop. She has striking black fox ears with fluffy fur, a black fox tail, and long black hair with red inner highlights. Her hair is adorned with a delicate ornament, and she wears a magatama necklace. Her outfit is a meticulously designed black short kimono with fur trim, featuring wide, long sleeves and an off-shoulder style. The kimono is paired with an obi and a miniskirt, and she also wears a cat collar. The girl is holding a donut and eating it, with a large cup of coffee on the table in front of her. The sketch is rendered with perfect and intricate detail, capturing the beauty and aesthetic of the scene, making it a true masterpiece.


BadRequestError: Error code: 400 - {'error': {'code': 'content_policy_violation', 'message': 'This request has been blocked by our content filters.', 'param': None, 'type': 'invalid_request_error'}}

### There is censorship in OpenAI...so I do not like it that much.

Every model has its strength
In my opinion:

- SDXL: style
- PONY: pose control, view angle control
- FLUX: realistic

In [10]:
dalle3_chain.invoke({"size": "1024x1024",
                     "quality": "hd",
                     "image_desc": """
                                 close-up portrait, black fox ears, animal ear fluff, black fox tail, black hair, red inner hair, hair ornament, 
                                 magatama necklace, fur trim, black short kimono, exquisite design, cat_collar, off-shoulder,wide sleeves, 
                                 long sleeves, obi, miniskirt, perfect model body, a 17-years-old ethereal and glamorously beautiful girl, from above, 
                                 eating donut, holding a donut, a large cup of coffee on table, in a coffee shop, pencil sketch, perfect detail, intricate detail, 
                                 masterpiece, best quality, beauty & aesthetic, sketch
                                 """,
                     "filename": "tutorial/Week-8/test_03.png"
                    })

Start generating image...
prompt: A breathtaking pencil sketch of an ethereal and glamorously beautiful 17-year-old girl with a perfect model body, captured from above in a coffee shop. She has striking black fox ears with fluffy fur, a black fox tail, and long, flowing black hair with red inner highlights. Her hair is adorned with a delicate ornament, and she wears a magatama necklace. Her attire is a black short kimono with an exquisite design, featuring off-shoulder wide sleeves and a fur trim. The kimono is paired with an obi and a miniskirt, showcasing her elegant and refined style. She also wears a cat collar, adding a touch of charm to her appearance. The girl is holding a donut and eating it, with a large cup of coffee placed on the table in front of her. The sketch is rendered with perfect and intricate detail, capturing the beauty and aesthetic of the scene, making it a true masterpiece.
Image is generated succesfully.


## Use this as a tool for Agent

In [11]:
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain.prompts import PromptTemplate
from langchain.tools import BaseTool
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

from src.agent.react_zero_shot import prompt_template as zero_shot_prompt_template

# We need both the query and filename (minimal requirement):
# Some control variables
# What we learned last week?


class ImageTool(BaseTool):

    name = "Image generator with DALLE-3"

    input_response_schemas = [
        ResponseSchema(name="image_desc", description="image description / prompt"),
        ResponseSchema(name="filename", description="the location at which the image will be saved"),
        ResponseSchema(name="size", description="image size, can be `1024x1024`, `1024x1792`, `1792x1024`"),
        ResponseSchema(name="quality", description="image quality, can be `hd` or `standard`"),
        ResponseSchema(name="style", description="image style, can be `vivid` or `natural`")]
    
    input_output_parser = StructuredOutputParser.from_response_schemas(input_response_schemas)
    
    input_format_instructions = input_output_parser.get_format_instructions()

    description_template = """
                           Use this tool when you need to create an image:
                           input format_instructions: {input_format_instructions}
                           """

    description = description_template.format(input_format_instructions=input_format_instructions)
    
    def _run(self, query):

        input_ = self.input_output_parser.parse(query)
        
        image_desc = input_['image_desc']
        size = input_['size']
        quality = input_['quality']
        style = input_['style']
        filename = input_['filename']
        
        dalle3_chain.invoke({"image_desc": image_desc,
                             "size": size,
                             "quality": quality,
                             "style": style,
                             "filename": filename})
        
        return "Done"

    def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")

# Zero Shot 標準模板
prompt = PromptTemplate.from_template(zero_shot_prompt_template)

# 建立工具庫 
tools = [ImageTool()]

# 創造Agent 
zero_shot_agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt,
)

# 創造Agent Executor
agent_executor = AgentExecutor(agent=zero_shot_agent, tools=tools, verbose=True)

In [12]:
image_prompt = """
brown hair, bangs, two side up, twin ponytails, sidelocks, black hat, jewelry, black cheongsam, intricate golden embroidery, long sleeves,
wide sleeves, black shorts, hat ornament, hat flower, a 17-years-old ethereal and glamorous beautiful japanese idol,
translucent skin tone, profound facial features, bright eyes, faint rosy blush, ultra realistic, raw photo, award-winning photo, masterpiece, 
best quality, high resolution, official art, 8k uhd, high fidelity, depth of field, on the top of a skyscaper, mesmetizing city view, 
night
"""

filename = "tutorial/Week-8/test_04.png"

In [13]:
agent_executor.invoke({"input": f"Generate in image with the following information: \n {image_prompt}. and save the image at {filename}"})



> Entering new AgentExecutor chain...
To generate the requested image, I will use the Image generator with DALLE-3 tool. I need to ensure that the image description is detailed and matches the provided specifications. The image will be saved at the specified location with the given filename.

Action: Image generator with DALLE-3

Action Input:
```json
{
	"image_desc": "A 17-year-old ethereal and glamorous beautiful Japanese idol with brown hair, bangs, two side up, twin ponytails, sidelocks, wearing a black hat with a hat ornament and hat flower, jewelry, and a black cheongsam with intricate golden embroidery, long wide sleeves, and black shorts. She has a translucent skin tone, profound facial features, bright eyes, and a faint rosy blush. The image is ultra-realistic, raw photo, award-winning photo, masterpiece, best quality, high resolution, official art, 8k uhd, high fidelity, depth of field. She is standing on the top of a skyscraper with a mesmerizing city view at night.",
	"fi

{'input': 'Generate in image with the following information: \n \nbrown hair, bangs, two side up, twin ponytails, sidelocks, black hat, jewelry, black cheongsam, intricate golden embroidery, long sleeves,\nwide sleeves, black shorts, hat ornament, hat flower, a 17-years-old ethereal and glamorous beautiful japanese idol,\ntranslucent skin tone, profound facial features, bright eyes, faint rosy blush, ultra realistic, raw photo, award-winning photo, masterpiece, \nbest quality, high resolution, official art, 8k uhd, high fidelity, depth of field, on the top of a skyscaper, mesmetizing city view, \nnight\n. and save the image at tutorial/Week-8/test_04.png',
 'output': 'The image has been generated and saved at the location `tutorial/Week-8/test_04.png`.'}

## ACG Characters



In [14]:
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.tools.tavily_search import TavilySearchResults

search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)

outputs = tavily_tool.invoke("What is the appearance of Hu Tao from Genshin")

In [16]:
TavilySearchAPIWrapper?

Init signature: TavilySearchAPIWrapper(*, tavily_api_key: pydantic.v1.types.SecretStr) -> None
Docstring:      Wrapper for Tavily Search API.
Init docstring:
Create a new model by parsing and validating input data from keyword arguments.

Raises ValidationError if the input data cannot be parsed to form a valid model.
File:           c:\users\mengchieh\miniconda3\envs\llm_examples\lib\site-packages\langchain_community\utilities\tavily_search.py
Type:           ModelMetaclass
Subclasses:     

In [15]:
len(outputs)

5

In [17]:
TavilySearchResults?

Init signature:
TavilySearchResults(
    *,
    name: str = 'tavily_search_results_json',
    description: str = 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.',
    args_schema: Type[pydantic.v1.main.BaseModel] = <class 'langchain_community.tools.tavily_search.tool.TavilyInput'>,
    return_direct: bool = False,
    verbose: bool = False,
    callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None,
    callback_manager: Optional[langchain_core.callbacks.base.BaseCallbackManager] = None,
    tags: Optional[List[str]] = None,
    metadata: Optional[Dict[str, Any]] = None,
    handle_tool_error: Union[bool, str, Callable[[langchain_core.tools.ToolException], str], NoneType] = False,
    handle_validation_error: Union[bool, str, Callable[[pydantic.v1.error_wrappers.ValidationEr

In [18]:
for output in outputs:
    print("\n**************************\n")
    print(output['content'])
    print("\n")
    


**************************

Hu Tao[Note 2] (Chinese: 胡桃 Hú Táo) is a playable Pyro character in Genshin Impact. Hu Tao's antics and eccentricity belies her role as the 77th Director of the Wangsheng Funeral Parlor and her talent as a poet. Nevertheless, she treats the parlor's operations with utmost importance, and holds funeral ceremonies with the highest dignity and solemnity. Toggle Ascension MaterialsTotal Cost



**************************

Hu Tao (Chinese: 胡桃; pinyin: Hú Táo; lit. 'Walnut') is a playable character in the action role-playing game Genshin Impact.She is voiced by Brianna Knickerbocker in English, Tao Dian [] in Chinese, Rie Takahashi in Japanese, and Kim Ha-ru [] in Korean. In the game, she serves as the 77th Director of the Wangsheng Funeral Parlor within the China-like nation of Liyue.



**************************

Appearance. Hu Tao's most immediately recognizable features are her porkpie hat adorned with a red plum branch with a large wooden talisman at its ce

# **** 預計第一個小時結束 ****

## LCEL ACG character appearance chain

In [19]:
from langchain.docstore.document import Document
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain.tools import BaseTool
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser


def tavily_content_parser(outputs):

    """
    Parses the content from a list of outputs into Document objects.
    
    Parameters:
    outputs (list): A list of dictionaries where each dictionary contains a 'content' key 
                    representing the content of a document.
    
    Returns:
    list: A list of Document objects, each initialized with the content from the 'outputs'.
    
    Example:
    >>> outputs = [
    ...     {'content': 'This is document 1 content.'},
    ...     {'content': 'This is document 2 content.'}
    ... ]
    >>> documents = tavily_content_parser(outputs)
    >>> for doc in documents:
    ...     print(doc.page_content)
    This is document 1 content.
    This is document 2 content.
    """
    
    documents = [Document(page_content=output['content']) for output in outputs]
    
    return documents


system_prompt = PromptTemplate.from_template('''You are a helpful AI assistant with deep knowledge of anime, manga, 
and mobile games. You will generate the face, body, attire, hairstyle, and accessories of an character in great 
detail with data provided from the `context` The output should look like:

 - Face:
 - Body:
 - Attire:
 - Hairstyle:
 - Accessories:

''')

system_message = SystemMessagePromptTemplate(prompt=system_prompt)


human_prompt = PromptTemplate(template="context: {context}",
                              input_variables=['context'])

# Create a human message prompt template based on the prompt
human_message = HumanMessagePromptTemplate(prompt=human_prompt)

# Create a chat prompt template from system and human message prompt templates
chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

# Construct the processing chain
step_1 = {'context': tavily_tool|tavily_content_parser}
step_2 = chat_prompt
step_3 = model
step_4 = StrOutputParser()
acg_chain = step_1 | step_2 | step_3 | step_4



In [20]:
output = acg_chain.invoke("What is the appearance of Hu Tao from Genshin")

In [21]:
print(output)

Based on the provided context, here is a detailed description of Hu Tao from Genshin Impact:

- **Face**:
  - Hu Tao has bright scarlet eyes that are one of her most striking features. Her eyes often have a blossom-like pattern within them, adding to her unique and mystical appearance.
  - Her face is youthful and expressive, often showing a playful or mischievous expression that belies her serious role.

- **Body**:
  - Hu Tao has a slender and agile build, fitting her role as a nimble and quick character in the game.
  - Her posture is often relaxed and confident, reflecting her eccentric and carefree personality.

- **Attire**:
  - Hu Tao wears a traditional Chinese-inspired outfit that is both elegant and functional. Her attire is primarily dark in color, with intricate red and gold patterns that symbolize her connection to the Pyro element.
  - She is often seen wearing a long, dark coat with red accents and a high collar, which adds to her mysterious and authoritative presence as

In [22]:
class ACGLLMTool(BaseTool):

    name = "`Anime character design generator`"
    description = "Use this tool to generate and explore detailed designs for anime and ACG (Animation, Comics, and Games) characters."

    def _run(self, query: str):
        
        description = acg_chain.invoke(query)
        
        return description

    def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")
        
        
class ImageTool(BaseTool):

    name = "ACG characters image generator with DALLE-3"

    input_response_schemas = [
        ResponseSchema(name="image_desc", description="image description / prompt"),
        ResponseSchema(name="filename", description="the location at which the image will be saved"),
        ResponseSchema(name="size", description="image size, can be `1024x1024`, `1024x1792`, `1792x1024`"),
        ResponseSchema(name="quality", description="image quality, can be `hd` or `standard`"),
        ResponseSchema(name="style", description="image style, can be `vivid` or `natural`")]
    
    input_output_parser = StructuredOutputParser.from_response_schemas(input_response_schemas)
    
    input_format_instructions = input_output_parser.get_format_instructions()

    description_template = """
                           This is a tool for creating images. 
                           It's best used when you're considering the need for an ACG (anime, comics, games) character design. 
                           Before using this tool, you may want to utilize the `Anime character design generator` to gather 
                           relevant information. The generated image will maintain the specified art style. 
                           input format: {input_format_instructions}
                           """

    description = description_template.format(input_format_instructions=input_format_instructions)
    
    def _run(self, query):
        
        input_ = self.input_output_parser.parse(query)
        
        image_desc = input_['image_desc']
        size = input_['size']
        quality = input_['quality']
        style = input_['style']
        filename = input_['filename']
        
        dalle3_chain.invoke({"image_desc": image_desc,
                             "size": size,
                             "quality": quality,
                             "style": style,
                             "filename": filename})
        
        return "Done"

    def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")

        
prompt_template = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer

Thought: you should always think about what to do

Action: the action to take, should be one of [{tool_names}]

Action Input: the input to the action

Observation: the result of the action

... (this Thought/Action/Action Input/Observation can repeat N times)

Thought: I now know the final answer

Final Answer: the final answer to the original input question

Begin!

Question: {input}

Thought:{agent_scratchpad}
"""        
             
prompt = PromptTemplate.from_template(prompt_template)

tools = [ImageTool(), ACGLLMTool()]

zero_shot_agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=zero_shot_agent, tools=tools, verbose=True)

In [23]:
agent_executor.invoke({"input": f"Generate an image of Hu Tao from Genshim in pastol art style"})



> Entering new AgentExecutor chain...
To generate an image of Hu Tao from Genshin Impact in a pastel art style, I should first gather detailed design information using the `Anime character design generator`. This will help ensure that the image generated by the `ACG characters image generator with DALLE-3` is accurate and meets the desired specifications.

Action: `Anime character design generator`

Action Input: Generate a detailed design for Hu Tao from Genshin Impact in a pastel art style.
Based on the provided context, here is a detailed description of the AI-generated Hu Tao character art from Genshin Impact:

- **Face**: 
  - Hu Tao's face is characterized by her youthful and mischievous expression. She has a fair complexion with a slight blush on her cheeks, giving her a lively and spirited look. Her eyes are a striking crimson red, often depicted with a playful glint that reflects her enigmatic personality. Her lips are small and slightly curved into a knowing smile, adding t

{'input': 'Generate an image of Hu Tao from Genshim in pastol art style',
 'output': 'The image of Hu Tao from Genshin Impact in a pastel art style has been generated successfully. The image features Hu Tao with a youthful and mischievous expression, fair complexion with a slight blush, striking crimson red eyes, and a knowing smile. She is depicted in a bikini with intricate flame patterns, predominantly red and gold colors, with a halter neck top and side-tie bottom. Her long dark brown hair cascades in loose waves with signature bangs and a traditional Chinese hairpin. She is accompanied by a ghostly companion and might have a small decorative fan or lantern, and a delicate anklet or bracelet with charms. The art style is pastel, and the image is in HD quality with a vivid style. The image is saved as "hu_tao_pastel_art_style.png" with a size of 1024x1024.'}

In [24]:
agent_executor.invoke({"input": "Generate an award-winning portrait photo of a 17-years-old japanese girl cosplaying Hu Tao from Genshim" })



> Entering new AgentExecutor chain...
To generate an award-winning portrait photo of a 17-year-old Japanese girl cosplaying Hu Tao from Genshin Impact, I need to first gather detailed design information about the character Hu Tao using the `Anime character design generator`. This will help ensure the cosplay is accurate and detailed.

Action: Anime character design generator

Action Input: Generate detailed design for Hu Tao from Genshin Impact
Anime character design generator is not a valid tool, try one of [ACG characters image generator with DALLE-3, `Anime character design generator`].It seems I made an error in my previous thought. I should directly use the `ACG characters image generator with DALLE-3` to create the image based on the known design of Hu Tao from Genshin Impact.

Action: ACG characters image generator with DALLE-3

Action Input: 
```json
{
	"image_desc": "A 17-year-old Japanese girl cosplaying as Hu Tao from Genshin Impact. She is wearing Hu Tao's traditional Chi

{'input': 'Generate an award-winning portrait photo of a 17-years-old japanese girl cosplaying Hu Tao from Genshim',
 'output': 'The award-winning portrait photo of a 17-year-old Japanese girl cosplaying Hu Tao from Genshin Impact has been generated and saved as "hu_tao_cosplay_portrait.png".'}

## Audible 有聲書

- 文轉語音: TTS tool
- 文轉圖: Image tool

### Children Book Image Generator

- Generate image according to the story

In [41]:
system_prompt = PromptTemplate.from_template('''You are a helpful AI assistant and an art expert with extensive knowledge of illustration. 
You excel at creating Pencil and Ink Style illustrations for 6-year-old children using the DALLE-3 model. This style is characterized by 
detailed line work, often in black and white or with minimal color, and has a classic, timeless feel. For this task, you will be provided with 
a paragraph of a story, and you will generate a corresponding DALLE-3 prompt which captures the storyline. The prompt should be 
detailed and descriptive, capturing the essence of the image. The length of the prompt should be around 100-500 tokens.''')

# System prompt
system_message = SystemMessagePromptTemplate(prompt=system_prompt)

human_prompt = PromptTemplate(template="{story}",
                              input_variables=['story'])

# Create a human message prompt template based on the prompt
human_message = HumanMessagePromptTemplate(prompt=human_prompt)

chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

nl_prompt_generation_chain = chat_prompt | model | StrOutputParser()     

step_1 = RunnablePassthrough.assign(nl_prompt=itemgetter('story')|nl_prompt_generation_chain)
step_2 = RunnablePassthrough.assign(image_base64=dalle3_worker)
step_3 = RunnableLambda(base64_to_file)
image_chain = step_1 | step_2 | step_3

In [ ]:
# step_1 = RunnablePassthrough.assign(nl_prompt=itemgetter('image_desc')|nl_prompt_generation_chain)

# # step 2: image generation process, as a base64
# step_2 = RunnablePassthrough.assign(image_base64=dalle3_worker)

# # step 3: save the image
# step_3 = RunnableLambda(base64_to_file)

# # chain step 1, step 2, step 3 together
# dalle3_chain =  step_1|step_2|step_3

- Generate the story

In [26]:
system_prompt = PromptTemplate.from_template('''You are a helpful AI assistant who likes children. 
You are great storyteller and know how to create content for kindergarten kids. A short chapter is 
created once at a time.''')

# System prompt
system_message = SystemMessagePromptTemplate(prompt=system_prompt)

human_prompt = PromptTemplate(template="{input}",
                              input_variables=['input'])

# Create a human message prompt template based on the prompt
human_message = HumanMessagePromptTemplate(prompt=human_prompt)

chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

story_chain = chat_prompt | model | StrOutputParser()     

In [42]:
import json

from src.agent.react_zero_shot import prompt_template as zero_shot_prompt_template
# from langchain_core.prompts import MessagesPlaceholder

client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

    
class TTSTool(BaseTool):
    
    name = "Text to Sound"
    description = "Use this tool to generate an audio file"

    name = "Text to Sound (tts) tool"

    input_response_schemas = [
        ResponseSchema(name="text", description="The story"),
        ResponseSchema(name="filename", description="the location at which the audio file will be saved")]
    
    input_output_parser = StructuredOutputParser.from_response_schemas(input_response_schemas)
    
    input_format_instructions = input_output_parser.get_format_instructions()

    description_template = """
                           Use this tool to generate an audio file of the story. 
                           input format: {input_format_instructions}
                           """

    description = description_template.format(input_format_instructions=input_format_instructions)

    
    def _run(self, text: str):

        input_ = self.input_output_parser.parse(text)

        text = input_['text']
        filename = input_['filename']
        
        response = self.tts(text)
        
        response.stream_to_file(filename)
        
        return filename

    def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")
        
        
    def tts(self, text: str):
        
        response = client.audio.speech.create(
          model="tts-1",
          voice="nova",
          input=text
        )

        return response
    
    
class ImageTool(BaseTool):

    name = "Image generator with DALLE-3"

    input_response_schemas = [
        ResponseSchema(name="image_desc", description="image description / prompt"),
        ResponseSchema(name="filename", description="the location at which the image will be saved"),
        ResponseSchema(name="size", description="image size, can be `1024x1024`, `1024x1792`, `1792x1024`"),
        ResponseSchema(name="quality", description="image quality, can be `hd` or `standard`"),
        ResponseSchema(name="style", description="image style, can be `vivid` or `natural`")]
    
    input_output_parser = StructuredOutputParser.from_response_schemas(input_response_schemas)
    
    input_format_instructions = input_output_parser.get_format_instructions()

    description_template = """
                           An image is generated and saved according to the input instruction.
                           input instruction:\n {input_format_instructions}
                           """

    description = description_template.format(input_format_instructions=input_format_instructions)
    
    def _run(self, query):
        
        input_ = self.input_output_parser.parse(query)
        
        image_desc = input_['image_desc']
        size = input_['size']
        quality = input_['quality']
        style = input_['style']
        filename = input_['filename']
        
        image_chain.invoke({"story": image_desc,
                             "size": size,
                             "quality": quality,
                             "style": style,
                             "filename": filename})
        
        return "Done"

    def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")
           
            
prompt = PromptTemplate.from_template(zero_shot_prompt_template)

tools = [TTSTool(), 
         ImageTool(),
         Tool(name="StoryTeller",
              func=story_chain.invoke,
              description="useful for create story",
        )]

zero_shot_agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=zero_shot_agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [43]:
prompt = """
        Create a chapter of a baby owl capturing a rodent in the night as his dinner. \
        After having the final answer, plrease create a corresponding image and record the story as an mp3. 
        The saved image and mp3 should have same name in the folder `tutorial/Week-8`
        """

agent_executor.invoke({"input": prompt})



> Entering new AgentExecutor chain...
I need to create a chapter of a baby owl capturing a rodent in the night as his dinner. After writing the story, I will generate an image and an audio file for the story. 

First, I will create the story.

Action: StoryTeller

Action Input: 
```json
{
  "input": "Create a chapter of a baby owl capturing a rodent in the night as his dinner."
}
```
**Chapter 1: Ollie the Baby Owl's Nighttime Adventure**

Once upon a time, in a cozy forest filled with tall trees and twinkling stars, there lived a baby owl named Ollie. Ollie had big, round eyes that sparkled like the moon and soft, fluffy feathers that kept him warm during the chilly nights.

One evening, as the sun dipped below the horizon and the sky turned a deep shade of blue, Ollie woke up from his nap. He stretched his tiny wings and blinked his sleepy eyes. It was time for his nighttime adventure!

"Whoo-whoo!" Ollie hooted softly, excited to explore the forest. He flapped his wings and soared

C:\Users\MengChieh\AppData\Local\Temp\ipykernel_17832\3449000546.py:41: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(filename)


tutorial/Week-8/Ollie_the_Baby_Owl.mp3I have successfully created the story, generated the corresponding image, and recorded the story as an mp3 file. Both the image and the audio file are saved in the folder `tutorial/Week-8` with the same name.

Final Answer: The chapter of the baby owl capturing a rodent in the night as his dinner has been created, along with the corresponding image and audio file. You can find them in the folder `tutorial/Week-8` with the names `Ollie_the_Baby_Owl.png` and `Ollie_the_Baby_Owl.mp3`.

> Finished chain.


{'input': '\n        Create a chapter of a baby owl capturing a rodent in the night as his dinner.         After having the final answer, plrease create a corresponding image and record the story as an mp3. \n        The saved image and mp3 should have same name in the folder `tutorial/Week-8`\n        ',
 'output': 'The chapter of the baby owl capturing a rodent in the night as his dinner has been created, along with the corresponding image and audio file. You can find them in the folder `tutorial/Week-8` with the names `Ollie_the_Baby_Owl.png` and `Ollie_the_Baby_Owl.mp3`.'}

## Can we create a story with multiple pages?

I do not know the answer, let me try...

4 pages to save the cost. But it can be extended.

In [44]:
prompt = """
         I want to create an 4 pages story for a child. He likes snow owl.
         For each page, please create a corresponding image and record the story as an mp3.
         After having the final answer, please create a corresponding image and record the story as an mp3. 
         The saved image and mp3 should have same name, following the structure of 
         <Page - idx>, with idx as a number starting from 1, in the folder `tutorial/Week-8`
         """

agent_executor.invoke({"input": prompt})



> Entering new AgentExecutor chain...
To create a 4-page story for a child who likes snow owls, I will first generate the story using the StoryTeller tool. Then, I will create corresponding images for each page using the Image generator with DALLE-3. Finally, I will record the story as an mp3 using the Text to Sound (tts) tool. Each page's image and audio file will be saved with the same name in the specified folder structure.

Let's start by generating the story.

Action: StoryTeller

Action Input: 
```json
{
  "input": "Create a 4-page story for a child who likes snow owls."
}
```
**Title: Ollie the Snow Owl's Winter Adventure**

**Page 1:**
Once upon a time, in a snowy forest far, far away, there lived a little snow owl named Ollie. Ollie had fluffy white feathers that helped him blend in with the snow. He loved to fly around the forest, exploring and making new friends. One chilly morning, Ollie woke up with a twinkle in his eye. Today was the day he would go on a big winter adve

C:\Users\MengChieh\AppData\Local\Temp\ipykernel_17832\3449000546.py:41: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(filename)


tutorial/Week-8/Page-1.mp3The first audio file has been generated successfully. Now, I will proceed to record the story for the second page.

Action: Text to Sound (tts) tool

Action Input: 
```json
{
	"text": "Ollie flapped his wings and soared high above the trees. As he flew, he saw his friend, Sammy the Squirrel, gathering acorns. 'Hi, Sammy!' Ollie hooted. 'I'm going on a winter adventure! Do you want to come with me?' Sammy's eyes sparkled with excitement. 'Yes, Ollie! Let's go!' So, Ollie and Sammy set off together, eager to see what wonders the snowy forest had in store.",
	"filename": "tutorial/Week-8/Page-2.mp3"
}
```


C:\Users\MengChieh\AppData\Local\Temp\ipykernel_17832\3449000546.py:41: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(filename)


tutorial/Week-8/Page-2.mp3The second audio file has been generated successfully. Now, I will proceed to record the story for the third page.

Action: Text to Sound (tts) tool

Action Input: 
```json
{
	"text": "As they ventured deeper into the forest, they came across a sparkling frozen pond. 'Look, Sammy! It's perfect for ice skating!' Ollie exclaimed. They found some smooth sticks to use as skates and glided across the ice, laughing and twirling. Suddenly, they heard a soft cry. 'Help! Help!' It was coming from a little snow bunny named Bella, who was stuck in a snowdrift.",
	"filename": "tutorial/Week-8/Page-3.mp3"
}
```


C:\Users\MengChieh\AppData\Local\Temp\ipykernel_17832\3449000546.py:41: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(filename)


tutorial/Week-8/Page-3.mp3The third audio file has been generated successfully. Now, I will proceed to record the story for the fourth page.

Action: Text to Sound (tts) tool

Action Input: 
```json
{
	"text": "Ollie and Sammy rushed over to help Bella. With a few strong flaps of his wings, Ollie cleared the snow away, and Sammy pulled Bella out. 'Thank you, Ollie and Sammy!' Bella said, her nose twitching with gratitude. 'You're welcome, Bella! Would you like to join our adventure?' Ollie asked. Bella nodded happily, and the three friends continued their journey, ready for more fun and surprises in the magical winter wonderland.",
	"filename": "tutorial/Week-8/Page-4.mp3"
}
```


C:\Users\MengChieh\AppData\Local\Temp\ipykernel_17832\3449000546.py:41: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(filename)


tutorial/Week-8/Page-4.mp3I now know the final answer.

Final Answer: The 4-page story for the child who likes snow owls has been successfully created. Each page has a corresponding image and an audio recording of the story. The files are saved in the folder `tutorial/Week-8` with the following structure:

- `tutorial/Week-8/Page-1.png`
- `tutorial/Week-8/Page-1.mp3`
- `tutorial/Week-8/Page-2.png`
- `tutorial/Week-8/Page-2.mp3`
- `tutorial/Week-8/Page-3.png`
- `tutorial/Week-8/Page-3.mp3`
- `tutorial/Week-8/Page-4.png`
- `tutorial/Week-8/Page-4.mp3`

Here is the story:

**Title: Ollie the Snow Owl's Winter Adventure**

**Page 1:**
Once upon a time, in a snowy forest far, far away, there lived a little snow owl named Ollie. Ollie had fluffy white feathers that helped him blend in with the snow. He loved to fly around the forest, exploring and making new friends. One chilly morning, Ollie woke up with a twinkle in his eye. Today was the day he would go on a big winter adventure!

**Page 

{'input': '\n         I want to create an 4 pages story for a child. He likes snow owl.\n         For each page, please create a corresponding image and record the story as an mp3.\n         After having the final answer, please create a corresponding image and record the story as an mp3. \n         The saved image and mp3 should have same name, following the structure of \n         <Page - idx>, with idx as a number starting from 1, in the folder `tutorial/Week-8`\n         ',
 'output': 'The 4-page story for the child who likes snow owls has been successfully created. Each page has a corresponding image and an audio recording of the story. The files are saved in the folder `tutorial/Week-8` with the following structure:\n\n- `tutorial/Week-8/Page-1.png`\n- `tutorial/Week-8/Page-1.mp3`\n- `tutorial/Week-8/Page-2.png`\n- `tutorial/Week-8/Page-2.mp3`\n- `tutorial/Week-8/Page-3.png`\n- `tutorial/Week-8/Page-3.mp3`\n- `tutorial/Week-8/Page-4.png`\n- `tutorial/Week-8/Page-4.mp3`\n\nHere is

## Can we create a story in an interactive way: chat based

-- Rolling back...

In [77]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

from langchain.output_parsers import StructuredOutputParser, ResponseSchema

output_response_schemas = [
        ResponseSchema(name="story", description="the story content in the page"),
        ResponseSchema(name="page index", description="The page number of the story"),
    ]

output_parser = StructuredOutputParser.from_response_schemas(output_response_schemas)

output_format_instructions = output_parser.get_format_instructions()


template = """
           Create a story page {idx}, based on the description: {text}

           Continuned from previous content:
           {context}

            
           After having the final answer, please create a corresponding image and record the story as an mp3. 
           The saved image and mp3 should have same name, following the structure of 
           <Page - idx>, in the folder `tutorial/Week-8`

           The output should have the following format: {output_format_instruction}
           """

prompt_template = PromptTemplate(template=template,
                                 input_variables=["text", "context", "idx"],
                                 partial_variables={"output_format_instruction": output_format_instructions})

agent_chain = RunnablePassthrough.assign(input=prompt_template)|agent_executor

In [71]:
Q = agent_chain.invoke({"text": "A little cat just woke up in the morning",
                        "context": "The beginning of the story:\n",
                        "idx": str(1)})



> Entering new AgentExecutor chain...
To create the story page based on the given description, I will first generate the story content for page 1. Then, I will create a corresponding image and record the story as an mp3 file. Finally, I will format the output as requested.

Let's start by generating the story content for page 1.

Action: StoryTeller
Action Input: "A little cat just woke up in the morning"
### Chapter 1: The Morning Adventure

Once upon a time, in a cozy little house, a tiny cat named Whiskers just woke up in the morning. Whiskers had soft, fluffy fur and the cutest little pink nose. As the sun peeked through the curtains, Whiskers stretched his paws and let out a big, happy yawn.

"Good morning, world!" Whiskers purred, feeling excited about the new day. He hopped out of his comfy bed and padded over to the window. Outside, the birds were chirping a cheerful song, and the flowers in the garden were blooming brightly.

Whiskers decided it was the perfect day for an ad

C:\Users\MengChieh\AppData\Local\Temp\ipykernel_17832\3449000546.py:41: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(filename)


tutorial/Week-8/Page-1.mp3The story content for page 1 has been generated, the corresponding image has been created, and the story has been recorded as an mp3 file. Now, I will format the output as requested.

Final Answer:
```json
{
	"story": "### Chapter 1: The Morning Adventure\n\nOnce upon a time, in a cozy little house, a tiny cat named Whiskers just woke up in the morning. Whiskers had soft, fluffy fur and the cutest little pink nose. As the sun peeked through the curtains, Whiskers stretched his paws and let out a big, happy yawn.\n\n\"Good morning, world!\" Whiskers purred, feeling excited about the new day. He hopped out of his comfy bed and padded over to the window. Outside, the birds were chirping a cheerful song, and the flowers in the garden were blooming brightly.\n\nWhiskers decided it was the perfect day for an adventure. He trotted into the kitchen, where his best friend, a little girl named Lily, was having breakfast.\n\n\"Good morning, Whiskers!\" Lily said with a s

若是以下步驟失敗，嘗試重新生成。這是大語言模型，沒有保證可以100%產出你希望的格式。我們只能盡可能提高成功輸出的機率。

In [72]:
Q

{'text': 'A little cat just woke up in the morning',
 'context': 'The beginning of the story:\n',
 'idx': '1',
 'input': StringPromptValue(text='\n           Create a story page 1, based on the description: A little cat just woke up in the morning\n\n           Previous content:\n           The beginning of the story:\n\n\n            \n           After having the final answer, please create a corresponding image and record the story as an mp3. \n           The saved image and mp3 should have same name, following the structure of \n           <Page - idx>, in the folder `tutorial/Week-8`\n\n           The output should have the following format: The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"story": string  // the story content in the page\n\t"page index": string  // The page number of the story\n}\n```\n           '),
 'output': '```json\n{\n\t"story": "### Chapter 1: The Morning

In [73]:
Q['output']

'```json\n{\n\t"story": "### Chapter 1: The Morning Adventure\\n\\nOnce upon a time, in a cozy little house, a tiny cat named Whiskers just woke up in the morning. Whiskers had soft, fluffy fur and the cutest little pink nose. As the sun peeked through the curtains, Whiskers stretched his paws and let out a big, happy yawn.\\n\\n\\"Good morning, world!\\" Whiskers purred, feeling excited about the new day. He hopped out of his comfy bed and padded over to the window. Outside, the birds were chirping a cheerful song, and the flowers in the garden were blooming brightly.\\n\\nWhiskers decided it was the perfect day for an adventure. He trotted into the kitchen, where his best friend, a little girl named Lily, was having breakfast.\\n\\n\\"Good morning, Whiskers!\\" Lily said with a smile, giving him a gentle pat on the head. \\"Are you ready for some fun today?\\"\\n\\nWhiskers meowed happily and rubbed against Lily\'s leg. He loved spending time with Lily, and he knew that every day wit

In [74]:
output_parser.parse(Q['output'])

{'story': '### Chapter 1: The Morning Adventure\n\nOnce upon a time, in a cozy little house, a tiny cat named Whiskers just woke up in the morning. Whiskers had soft, fluffy fur and the cutest little pink nose. As the sun peeked through the curtains, Whiskers stretched his paws and let out a big, happy yawn.\n\n"Good morning, world!" Whiskers purred, feeling excited about the new day. He hopped out of his comfy bed and padded over to the window. Outside, the birds were chirping a cheerful song, and the flowers in the garden were blooming brightly.\n\nWhiskers decided it was the perfect day for an adventure. He trotted into the kitchen, where his best friend, a little girl named Lily, was having breakfast.\n\n"Good morning, Whiskers!" Lily said with a smile, giving him a gentle pat on the head. "Are you ready for some fun today?"\n\nWhiskers meowed happily and rubbed against Lily\'s leg. He loved spending time with Lily, and he knew that every day with her was full of surprises.\n\nAfte

In [75]:
output_parser.parse(Q['output'])['story']

'### Chapter 1: The Morning Adventure\n\nOnce upon a time, in a cozy little house, a tiny cat named Whiskers just woke up in the morning. Whiskers had soft, fluffy fur and the cutest little pink nose. As the sun peeked through the curtains, Whiskers stretched his paws and let out a big, happy yawn.\n\n"Good morning, world!" Whiskers purred, feeling excited about the new day. He hopped out of his comfy bed and padded over to the window. Outside, the birds were chirping a cheerful song, and the flowers in the garden were blooming brightly.\n\nWhiskers decided it was the perfect day for an adventure. He trotted into the kitchen, where his best friend, a little girl named Lily, was having breakfast.\n\n"Good morning, Whiskers!" Lily said with a smile, giving him a gentle pat on the head. "Are you ready for some fun today?"\n\nWhiskers meowed happily and rubbed against Lily\'s leg. He loved spending time with Lily, and he knew that every day with her was full of surprises.\n\nAfter breakfas

In [59]:
output_parser.parse(Q['output'])['page index']

'1'

### 第二頁

In [79]:
context_list = [output_parser.parse(Q['output'])['story']]
print(context_list)

['### Chapter 1: The Morning Adventure\n\nOnce upon a time, in a cozy little house, a tiny cat named Whiskers just woke up in the morning. Whiskers had soft, fluffy fur and the cutest little pink nose. As the sun peeked through the curtains, Whiskers stretched his paws and let out a big, happy yawn.\n\n"Good morning, world!" Whiskers purred, feeling excited about the new day. He hopped out of his comfy bed and padded over to the window. Outside, the birds were chirping a cheerful song, and the flowers in the garden were blooming brightly.\n\nWhiskers decided it was the perfect day for an adventure. He trotted into the kitchen, where his best friend, a little girl named Lily, was having breakfast.\n\n"Good morning, Whiskers!" Lily said with a smile, giving him a gentle pat on the head. "Are you ready for some fun today?"\n\nWhiskers meowed happily and rubbed against Lily\'s leg. He loved spending time with Lily, and he knew that every day with her was full of surprises.\n\nAfter breakfa

In [80]:
Q_2 = agent_chain.invoke({"text": "Whisker found a dove and wanted to hunt it down!",
                          "context": ":\n".join(context_list),
                          "idx": str(2)})



> Entering new AgentExecutor chain...
To create the second page of the story based on the given description, I will first generate the continuation of the story. Then, I will create an image and an audio file for the story. Finally, I will provide the output in the required format.

Let's start by generating the continuation of the story.

Action: StoryTeller
Action Input: "Whisker found a dove and wanted to hunt it down! Continued from previous content: ### Chapter 1: The Morning Adventure Once upon a time, in a cozy little house, a tiny cat named Whiskers just woke up in the morning. Whiskers had soft, fluffy fur and the cutest little pink nose. As the sun peeked through the curtains, Whiskers stretched his paws and let out a big, happy yawn. 'Good morning, world!' Whiskers purred, feeling excited about the new day. He hopped out of his comfy bed and padded over to the window. Outside, the birds were chirping a cheerful song, and the flowers in the garden were blooming brightly. Wh

C:\Users\MengChieh\AppData\Local\Temp\ipykernel_17832\3449000546.py:41: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(filename)


tutorial/Week-8/Page-2.mp3I have successfully generated the audio file for the second page of the story. Now, I will provide the final output in the required format.

Final Answer:
```json
{
	"story": "### Chapter 2: The Curious Dove\n\nWhiskers and Lily wandered through the garden, their eyes wide with wonder. The flowers were in full bloom, painting the garden with splashes of red, yellow, and purple. Butterflies fluttered around, and bees buzzed busily from flower to flower.\n\nAs they walked, Whiskers' sharp eyes caught sight of something white and fluttery in the distance. It was a beautiful dove, perched on a low branch of the apple tree. The dove's feathers were as white as snow, and it cooed softly, adding to the peaceful morning sounds.\n\nWhiskers' curiosity was piqued. He crouched low, his little blue hat almost slipping off his head, and began to creep towards the dove. His tail swished back and forth, and his eyes were fixed on the bird.\n\nLily noticed Whiskers' intent lo

In [69]:
context_list

['the beginning of the story, page 1',
 "Once upon a time, in a cozy little house, there lived a tiny, fluffy cat named Whiskers. One sunny morning, Whiskers woke up with a big stretch and a soft yawn. The sun was shining through the window, making everything look warm and golden.\n\nWhiskers loved mornings because they were full of new adventures. Today, he felt extra excited. He hopped out of his comfy bed and padded over to the window. Outside, the birds were singing cheerful songs, and the flowers in the garden were blooming with bright colors.\n\nWhiskers decided it was the perfect day to explore. He trotted over to his favorite spot in the kitchen, where his bowl of milk was waiting for him. After lapping up the delicious milk, he felt ready to start his day.\n\nAs Whiskers stepped outside, he noticed a butterfly fluttering by. It had beautiful, colorful wings that sparkled in the sunlight. Whiskers' eyes widened with curiosity. 'I wonder where the butterfly is going,' he thought

In [81]:
output_parser.parse(Q_2['output'])['story']

"### Chapter 2: The Curious Dove\n\nWhiskers and Lily wandered through the garden, their eyes wide with wonder. The flowers were in full bloom, painting the garden with splashes of red, yellow, and purple. Butterflies fluttered around, and bees buzzed busily from flower to flower.\n\nAs they walked, Whiskers' sharp eyes caught sight of something white and fluttery in the distance. It was a beautiful dove, perched on a low branch of the apple tree. The dove's feathers were as white as snow, and it cooed softly, adding to the peaceful morning sounds.\n\nWhiskers' curiosity was piqued. He crouched low, his little blue hat almost slipping off his head, and began to creep towards the dove. His tail swished back and forth, and his eyes were fixed on the bird.\n\nLily noticed Whiskers' intent look and giggled. 'Whiskers, are you trying to make a new friend?' she asked, following him quietly.\n\nWhiskers didn't answer; he was too focused on the dove. He moved closer and closer, his paws barely

### okay, it looks fine, let us see how to make it a interactive

In [ ]:
# "前情提要"
context_list = []

# 頁面起始
idx = 1

while True:
    if len(context_list) == 0:
        context = "The beginning of the story:\n"
    else:
        context = "\n".join(context_list)

    text = input("請輸入故事內容: 若想要結束 請輸入 `QUIT`")

    if text == "QUIT":
        break
    
    Q = agent_chain.invoke({"text": text,
                            "context": context,
                            "idx": str(idx)})

    story = output_parser.parse(Q['output'])['story']
    
    # 下一頁
    idx += 1

    context_list.append(output_parser.parse(Q['output'])['story'])

請輸入故事內容: 若想要結束 請輸入 `QUIT` A little cat just woke up in the morning




> Entering new AgentExecutor chain...
To create the story page based on the given description, I need to generate the story content first. Then, I will create an image and an audio file for the story. Finally, I will format the output as specified.

Let's start by creating the story content for page 1.

Action: StoryTeller

Action Input: "A little cat just woke up in the morning"
### Chapter 1: The Morning Adventure

Once upon a time, in a cozy little house, a tiny cat named Whiskers just woke up in the morning. Whiskers had soft, fluffy fur that was as white as snow, and big, curious blue eyes that sparkled like the morning dew.

As the first rays of sunshine peeked through the window, Whiskers stretched his little paws and let out a big, happy yawn. "Today is going to be a wonderful day!" he thought to himself. He could hear the birds chirping outside and the gentle rustle of leaves in the breeze.

Whiskers hopped out of his comfy bed and padded over to the window. He loved looking

C:\Users\MengChieh\AppData\Local\Temp\ipykernel_17832\3449000546.py:41: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(filename)


tutorial/Week-8/Page-1.mp3I have successfully generated the image and recorded the story as an mp3 file. Now, I will format the output as specified.

Final Answer:
```json
{
	"story": "### Chapter 1: The Morning Adventure\n\nOnce upon a time, in a cozy little house, a tiny cat named Whiskers just woke up in the morning. Whiskers had soft, fluffy fur that was as white as snow, and big, curious blue eyes that sparkled like the morning dew.\n\nAs the first rays of sunshine peeked through the window, Whiskers stretched his little paws and let out a big, happy yawn. 'Today is going to be a wonderful day!' he thought to himself. He could hear the birds chirping outside and the gentle rustle of leaves in the breeze.\n\nWhiskers hopped out of his comfy bed and padded over to the window. He loved looking outside to see what adventures awaited him. In the garden, he saw his best friend, a cheerful little bird named Tweet, fluttering around.\n\n'Good morning, Tweet!' Whiskers called out.\n\n'Good

請輸入故事內容: 若想要結束 請輸入 `QUIT` A shining light bursts out of the flower and sends them into the world of warhammer 40k. 


...

tutorial/Week-8/Page-2.mp3I have generated the continuation of the story, created the corresponding image, and recorded the story as an mp3. Now, I will provide the output in the required format.

```json
{
	"story": "### Chapter 2: The Shining Light\n\nWhiskers and Tweet wandered deeper into the garden, their eyes scanning every nook and cranny for the magical flower. After a while, they stumbled upon a hidden corner of the garden where the sunlight seemed to dance on the petals of a single, extraordinary flower.\n\n'There it is!' Tweet chirped excitedly. 'The magical flower!'\n\nWhiskers approached the flower cautiously, his eyes wide with wonder. The petals sparkled like stars, just as Tweet had described. As Whiskers reached out to touch one of the petals, a brilliant light burst forth from the flower, enveloping both Whiskers and Tweet.\n\nBefore they could react, the light lifted them off the ground and swirled around them, faster and faster, until they felt like they were being 

請輸入故事內容: 若想要結束 請輸入 `QUIT` They joined the adeptus custodes and serve the god emperor for the rest of their lives.




> Entering new AgentExecutor chain...
To create the story page 3 based on the given description, I need to continue the story from where it left off and incorporate the new elements about joining the Adeptus Custodes and serving the God Emperor. After writing the story, I will generate an image and an audio file for the story.

Action: StoryTeller

Action Input: 
```json
{
  "input": "### Chapter 3: The Oath of Service\n\nWhiskers and Tweet stood in awe as the towering figures in massive, armored suits introduced themselves as the Adeptus Custodes, the elite guardians of the God Emperor. The Custodes explained that Whiskers and Tweet had been chosen to join their ranks and serve the God Emperor for the rest of their lives.\n\n'But we're just a cat and a bird,' Whiskers said, his voice filled with uncertainty. 'How can we possibly serve the God Emperor?'\n\nThe leader of the Custodes, a stern figure with a golden helmet, knelt down to their level. 'It is not your form that matters, bu

C:\Users\MengChieh\AppData\Local\Temp\ipykernel_17832\3449000546.py:41: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(filename)


tutorial/Week-8/Page-3.mp3I have successfully generated both the image and the audio file for page 3 of the story. Now, I will provide the final answer in the required format.

Final Answer:
```json
{
	"story": "### Chapter 3: The Oath of Service\n\nWhiskers the cat and Tweet the bird were amazed as they looked up at the tall, shiny figures in big, golden suits. These figures were the Adeptus Custodes, the special protectors of the God Emperor. The Custodes told Whiskers and Tweet that they had been chosen to join their team and help protect the God Emperor forever.\n\n'But we're just a cat and a bird,' Whiskers said, feeling a bit scared. 'How can we help the God Emperor?'\n\nThe leader of the Custodes, who wore a shiny golden helmet, bent down to talk to them. 'It's not about what you look like, but about your brave hearts,' he said kindly. 'The God Emperor believes everyone has something special inside them, no matter how big or small they are.'\n\nThe Custodes then took Whiskers an